In [2]:
import numpy as np
import numpy.linalg as LA

In [2]:
def V_z(y):
  raise NotImplementedError

In [3]:
def T(y):
  raise NotImplementedError

In [4]:
def M_x(y):
  raise NotImplementedError

In [ ]:
b2 = 16.07/2
root_c = np.nan 
tip_c = np.nan

def chord_length(y):
    """Returns chord length at spanwise posistion"""
    return root_c + y * (tip_c - root_c)/b2

def airfoil_surface(cls, x):
    """Returns upper and lower coordinate of airfoil surface for a chordwise posistion"""
    raise NotImplementedError
    return z_upper, z_lower

In [ ]:
class Panel:

    def __init__(self, point1:np.array, point2:np.array, thickness):
        """point1 and point2 are coordinates relative to the chord, thickness in mm"""
        self.p1 = point1
        self.p2 = point2
        self.vector = point2 - point1
        self.h = LA.norm(self.vector)
        self.t = thickness/1000
        # z is the vertical distance from the x axis relative to chord
        self.z = (point1 + self.vector/2)[0]

    def A(self, chord):
        return self.h * chord * self.t
    
    def Q_x(self, chord):
        """Returns first moment of area about z=0"""
        return self.z * chord * self.area(chord)
    
    def I_xc(self, chord):
        """Returns second moment of area about own axis"""
        cos_a = self.vector[0] / self.h
        sin_a = self.vector[1] / self.h

        return self.t * self.h * (self.t**2 * cos_a**2 + self.h**2 * sin_a**2)
    
    def I_xc(self, chord, z_centroid):
        """Returns second moment of area about centroid"""
        d = self.z * chord - z_centroid
        return self.I_xc(chord) + self.area(chord) * d**2
    


In [9]:
class WingBox:

  def __init__(self, front_spar_x, front_spar_t, rear_spar_x, rear_spar_t, upper_panel_t, lower_panel_t, upper_stringers=None, 
                lower_stringers=None):
    front_upper_z, front_lower_z = airfoil_surface(front_spar_x)
    self.front_upper, self.front_lower = np.array([front_spar_x, front_upper_z]), np.array([front_spar_x, front_lower_z])
    rear_upper_z, rear_lower_z = airfoil_surface(rear_spar_x)
    self.rear_upper, self.rear_lower = np.array([rear_spar_x, rear_upper_z]), np.array([rear_spar_x, rear_lower_z])

    front_spar = Panel(self.front_lower, self.front_upper, front_spar_t)
    rear_spar = Panel(self.rear_lower, self.rear_upper, rear_spar_t)
    upper_panel = Panel(self.front_upper, self.rear_upper, upper_panel_t)
    lower_panel = Panel(self.front_lower, self.rear_lower, lower_panel_t)

    self.panels = (front_spar, rear_spar, upper_panel, lower_panel)
  
    self.upper_stringers = upper_stringers
    self.lower_stringer = lower_stringers

  def z_centroid(self, y):
    """Returns the z coordinate of the centroid of the cross section at a spanwise posistion"""
    # # First moment of area is calculated as Sum Az / Sum A
    # # vectors that represent the 
    # c = chord_length(y)
    # # represent the panels as vectors
    # front_spar = self.front_spar * c
    # rear_spar = self.rear_spar * c
    # upper_plate = self.upper_plate * c
    # lower_plate = self.lower_plate * c

    # front_spar_A = LA.norm(front_spar) * self.front_spar_t
    # front_spar_z = front_spar/2 + self.front_lower
    # rear_spar_A = LA.norm(rear_spar) * self.rear_spar_t
    # rear_spar_z = rear_spar/2 + self.rear_lower
    # upper_plate_A = LA.norm(upper_plate) * self.upper_plate_t
    # upper_plate_z = upper_plate/2 + self.front_upper
    # lower_plate_A = LA.norm(lower_plate) * self.lower_plate_t
    # lower_plate_z = lower_plate/2 + self.lower_plate_t

    # Az = front_spar_A*front_spar_z + rear_spar_A*rear_spar_z + upper_plate_A*upper_plate_z + lower_plate_A*lower_plate_z
    # A = front_spar_A + rear_spar_A + upper_plate_A + lower_plate_A
    
    # for stringer in self.upper_stringers:
    #   stringer_A, stringer_x, stringer_end, offset = stringer
    #   # making the stringer x coordinate relative to the front spar and multiplying by the chord
    #   stringer_x = (self.front_upper[0] - stringer_x)*c
    #   if stringer_end > y:
    #     pass
    #   stringer_z = self.front_upper + upper_plate[1] * stringer_x/upper_plate[0] + offset * upper_plate[0]/LA.norm(upper_plate)
    #   Az += stringer_A * stringer_z
    #   A += stringer_A

    # for stringer in self.lower_stringers:
    #   stringer_A, stringer_x, stringer_end, offset = stringer
    #   # making the stringer x coordinate relative to the front spar and multiplying by the chord
    #   stringer_x = (self.front_upper[0] - stringer_x)*c
    #   if stringer_end > y:
    #     pass
    #   stringer_z = self.front_upper + lower_plate[1] * stringer_x/lower_plate[0] + offset * lower_plate[0]/LA.norm(lower_plate)
    #   Az += stringer_A * stringer_z
    #   A += stringer_A

    # return Az/A
    pass

  def I_xx(self, y):
    c = chord_length(y)
    z = self.z_centroid(y)
    A = 0
    Az = 0

    for panel in self.panels:
      

  



  def J(self, y):
    raise NotImplementedError

In [1]:
stringers = np.array([["area", "chord", "span", "offset"], ["area", "chord", "span"]])
stringers

NameError: name 'np' is not defined

2.5